In [20]:
from collections import defaultdict
from django.db.models.signals import *
import pandas as pd
from itertools import chain
from tqdm.notebook import tqdm
import datetime
import calendar
import numpy as np
from django.db.models import Q
import pytz

In [21]:
def normalize_month(month, year):
    return month + 12*(year - 2020)

In [22]:
normalize_month(3, 2024)

51

In [23]:
def is_valid(cnpj):
    try:
        cnpj = int(cnpj.replace('.', '').replace('/', '').replace('-', ''))
        return True
    except:
        return False

In [24]:
def users_ativos_inativos(query,data_inicio,data_fim):
    counterAtivo,counterInativo=0,0
    userAtivo=[]
    userInativo=[]
    uic=query.order_by('user','-history_date').distinct('user')
    for i,a in enumerate(uic):
        try: # Try necessário pois teve usuário que foi deletado e da problema na geração do script
            inactivated_this_month=query.filter(is_active=False,
                                                user=a.user,
                                                history_date__gte=data_inicio,
                                                history_date__lte=data_fim).exists()
            history_date = a.history_date
            if history_date>data_inicio:
                resto=(history_date - data_inicio).days
            else:
                resto=(data_inicio - history_date).days

            if a.is_active==False:
                
                if inactivated_this_month:
                    if a.user.username not in [c[3] for c in userAtivo]:
                        resto=(a.history_date - data_inicio).days
                        userAtivo.append((a.user.get_full_name(),a.permissions.name,resto,a.user.username))
#                     else:
#                         print('repetido',a.user.username,a.user.get_full_name())
                        counterAtivo+=1
                else:
                    if a.user.username not in [c[3] for c in userInativo]:
                        userInativo.append((a.user.get_full_name(),a.permissions.name,"",a.user.username))
                        counterInativo+=1
    #                 print('pulou',a.user.get_full_name())
            else:
                if a.user.username not in [c[3] for c in userAtivo]:
                    userAtivo.append((a.user.get_full_name(),a.permissions.name,resto,a.user.username))
#                 else:
#                     print('repetido',a.user.username,a.user.get_full_name())
                counterAtivo+=1
        except Exception as e:
            pass
    return userAtivo,userInativo

In [25]:
def permissoes_history_user(result_list,data_inicio,data_fim):
    temp_ativo,temp_inativo,usuariosAtivos,usuariosInativos=[],[],[],[]
        
    companies = list(result_list.values_list('company__name',flat=True).order_by('company__name').distinct('company__name'))
    
    # condições para tratar dos casos de agrupamento de unidades
    if len(companies) == 1:
        usuariosAtivos,usuariosInativos=users_ativos_inativos(result_list,data_inicio,data_fim)
    
    # se há mais de uma unidade quer dizer q foi feito um agrupamentos por unidade e temos que
    # iterar sobre cada unidade separadamente, pra no fim retornar a lista de usuários por causa de um erro de o usuário
    # estar como inativo numa e ativo na outra
    elif len(companies) > 1:
        
        for company in companies:
            result_list_filtered = result_list.filter(company__name=company)
            
            usuariosAtivos,usuariosInativos=users_ativos_inativos(result_list_filtered,data_inicio,data_fim)
            
            temp_ativo=users_ativos_inativos(result_list_filtered,data_inicio,data_fim)[0] + temp_ativo
            temp_inativo=users_ativos_inativos(result_list_filtered,data_inicio,data_fim)[1] + temp_inativo
            
        try:
            usuariosInativos = list(set([x for x in temp_inativo if x not in temp_ativo]))
            usuariosAtivos = list(set(temp_ativo))
        except:
            print(usuariosAtivos,usuariosInativos)
    return usuariosAtivos,usuariosInativos

In [26]:
# Mês e ano para coleta dos usuários
mes=5
ano=2024
dia = calendar.monthrange(ano, mes)[1]
dia,mes,ano

(31, 5, 2024)

In [27]:
# DATA sempre referente ao mês que se quer coletar nesse formato history_date__lte='2024-01-31 03:00:00'
# é utilizado 3 horas da manha pois é a pra fechar com o horario de meio noite do servidor
utc=pytz.UTC
today_date = timezone.now()
data_fim = datetime.datetime(ano,mes,dia,3,0,0).replace(tzinfo=utc)
data_inicio = datetime.datetime(ano,mes,1).replace(tzinfo=utc)
data_fim,data_inicio

(datetime.datetime(2024, 5, 31, 3, 0, tzinfo=<UTC>),
 datetime.datetime(2024, 5, 1, 0, 0, tzinfo=<UTC>))

In [28]:
cnpjs=[a for a in Company.objects.filter(
    active=True).exclude(
    cnpj='00000000000000').exclude(name='Entrevias').distinct(
    'cnpj').order_by().values_list(
    'cnpj',flat=True)]
companies=cnpjs+[a for a in Company.objects.filter(
    active=True,cnpj='00000000000000').exclude(
    name__icontains='kartado').exclude(
    name='Entrevias').order_by(
    'name').values_list(
    'name',flat=True)]+['Entrevias']

In [29]:
# companies=['Arteris Via Paulista']

In [30]:
filename='Usuários Ativos Staging'
pd.DataFrame().to_excel(filename+'.xlsx')

In [31]:
for company in tqdm(companies):
    planilha_name=company.replace('.', '').replace('/', '').replace('-', '')
    if is_valid(company):
        # Agrupa por CNPJ
        queryHistory = UserInCompany.history.filter(
                company__cnpj=company,
                history_date__lte=data_fim
            ).exclude(
                user__email__icontains='kartado'
            ).exclude(
                user__email__icontains='roadlabs'
            ).exclude(
                user__email__icontains='hermes'
            ).order_by('user','-history_date')
    elif 'Entrevias' in company:
         queryHistory = UserInCompany.history.filter(
                 Q(user__email__icontains='afirma')|Q(user__email__icontains='ajr')|Q(user__username__icontains='ajr'),
                company__name=company,
                history_date__lte=data_fim
            ).exclude(
                user__email__icontains='kartado'
            ).exclude(
                user__email__icontains='roadlabs'
            ).exclude(
                user__email__icontains='hermes'
            ).order_by('user','-history_date')
    else:
         queryHistory = UserInCompany.history.filter(
                company__name=company,
                history_date__lte=data_fim
            ).exclude(
                user__email__icontains='kartado'
            ).exclude(
                user__email__icontains='roadlabs'
            ).exclude(
                user__email__icontains='hermes'
            ).order_by('user','-history_date')
#     print(len(queryHistory))
    users_permissions = permissoes_history_user(queryHistory,data_inicio,data_fim)[0] if len(permissoes_history_user(queryHistory,data_inicio,data_fim)[0]) > 0 else 0
    
    if users_permissions != 0:
        df2 = pd.DataFrame(data=users_permissions,columns=['Usuários ativos','Permissão','Dias desde a ultima solicitação',"Username"])
        companies_list=queryHistory.distinct('company__name').order_by( 'company__name').values_list('company__name',flat=True)
        print(company,companies_list)
        df2['Companies'] = np.nan
        df2.loc[:len(companies_list)-1, 'Companies'] = companies_list
        with pd.ExcelWriter(filename+'.xlsx', mode='a',if_sheet_exists='replace') as writer:
            df2.to_excel(writer,sheet_name=planilha_name,na_rep="-",index=False)

  0%|          | 0/36 [00:00<?, ?it/s]

1
00.000.000/0000-00 <QuerySet ['Teste - Inteligência']>
151
02.474.103/0008-95 <QuerySet ['OLD UHE-PASSO FUNDO']>
124
02.474.103/0009-76 <QuerySet ['OLD UHE-SALTO OSORIO']>
409
02474103/0012-71 <QuerySet ['GEH', 'OLD UHE-ITA', 'PCH-JOSÉ GELÁZIO DA ROCHA', 'PCH-RONDONÓPOLIS', 'POC-Projeto APR', 'Porto Itapoá', 'Projeto Formulários Arteris', 'SETA', 'UHE-CANA BRAVA', 'UHE-ESTREITO', 'UHE-ITA', 'UHE-JAGUARA', 'UHE-MACHADINHO', 'UHE-MIRANDA', 'UHE-PASSO FUNDO', 'UHE-PONTE DE PEDRA', 'UHE-SALTO OSORIO', 'UHE-SALTO SANTIAGO', 'UHE-SÃO SALVADOR']>
70
02.474.103/0014-33 <QuerySet ['OLD UHE-CANA BRAVA']>
63
02.474.103/0018-67 <QuerySet ['OLD UHE-SÃO SALVADOR']>
140
02.474.103/0019-48 <QuerySet ['OLD PCH-JOSÉ GELÁZIO DA ROCHA', 'OLD PCH-RONDONÓPOLIS', 'OLD UHE-PONTE DE PEDRA']>
118
02.955.426/0001-24 <QuerySet ['Campina Grande', 'ELO Teste', 'Neovia Campo Mourão 213/2019', 'Neovia Planalto Sul', 'Neovia Regis', 'Neovia Tijucas']>
330
02.955.426/0002-05 <QuerySet ['Neovia Araquari 886/2020', 'Ne

In [32]:
df2

,Usuários ativos,Permissão,Dias desde a ultima solicitação,Username,Companies
0,Humberto Fornari,ADMINISTRATOR,1472,humberto.fornari,Vig. Epidemiológica Brusque
1,Bernardo Przysiezny,ADMINISTRATOR,1461,bernardo.przysiezny,NaN
2,Caroline Maçaneiro,ADMINISTRATOR,1471,carol.epidemiologia,NaN
3,Alícia Fagundes,ADMINISTRATOR,1455,alicia.fagundes,NaN


In [33]:
data=[]
for a in UserInCompany.objects.filter(company__name=company):
    if a.user.username not in data:
        data.append(a.user.username)
    else:
        print(a.user.username)

In [161]:
str(uic.query) #para ver a query utilizada

'SELECT "companies_userincompany"."uuid", "companies_userincompany"."user_id", "companies_userincompany"."company_id", "companies_userincompany"."expiration_date", "companies_userincompany"."level", "companies_userincompany"."permissions_id", "companies_userincompany"."added_permissions", "companies_userincompany"."is_active" FROM "companies_userincompany" INNER JOIN "companies_company" ON ("companies_userincompany"."company_id" = "companies_company"."uuid") WHERE "companies_company"."name" = Arteris Via Paulista ORDER BY "companies_userincompany"."company_id" ASC'

In [24]:
# for a in UserInCompany.history.filter(company__name='Construtora Castilho (Prefeitura Curitiba)',
#                              user__username='macksson.araujo'):
#     print(a.__dict__)

{'_state': <django.db.models.base.ModelState object at 0x7d4886bc2250>, 'uuid': UUID('2be70483-7f76-483f-9798-777a254b1f66'), 'expiration_date': None, 'level': 2, 'added_permissions': None, 'user_id': UUID('b54024d2-7ac7-4387-847d-e56404803266'), 'company_id': UUID('837181f0-5c09-42a6-910d-e799607b8701'), 'permissions_id': UUID('79044f8c-a5aa-4ae6-9ab9-61fc92f2f01b'), 'history_id': UUID('00d232d0-0515-49f1-b24f-c8ee76f7d787'), 'history_date': datetime.datetime(2023, 11, 21, 16, 14, 41, 192993, tzinfo=<UTC>), 'history_change_reason': None, 'history_type': '-', 'history_user_id': UUID('6fcf5505-d2de-4ef1-97fe-81803e3b04a6')}
{'_state': <django.db.models.base.ModelState object at 0x7d4864f312e0>, 'uuid': UUID('2be70483-7f76-483f-9798-777a254b1f66'), 'expiration_date': None, 'level': 2, 'added_permissions': None, 'user_id': UUID('b54024d2-7ac7-4387-847d-e56404803266'), 'company_id': UUID('837181f0-5c09-42a6-910d-e799607b8701'), 'permissions_id': UUID('79044f8c-a5aa-4ae6-9ab9-61fc92f2f01b')